In [ ]:
# pairwise GSM implementation: shared and independent
# author: Amir Farzmahdi
# last update: Sep 16th, 2024

In [2]:
# library imports
import os
import random
import pickle
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pymc3 as pm
from scipy.stats import pearsonr
import pandas as pd

In [ ]:
# set random seed for NumPy and Python's random module
random.seed(42)
np.random.seed(42)

In [ ]:
# dictionary of changable part of settings for each figure and panel
settings = {
    
    # Supplementary Figure 3
    "figureS3A_shared": {
        "model_type": "shared",
        "date_train": "2024_05_01",
        "date_test": "2024_05_02",
        "cs_lev": [1, 1], # level 2
        "ntest": 500,
        "imageset": 'bsd500',
        "dim": 2,
        "n_theta": 9,
        "n_row": 9,
        "n_col": 9,
        "tcov_loc": 40,
        "ncov_loc": 40,
        "tr_cov_scale": 1,
        "noise_cov_scale_pair": 0.5,
        "noise_cov_scale_single": 1,
        "ndraw": 500,
        "ntune": 500,
        "ncov_type": 'overlapped',
        "dc_offset": 2,
    },
    
    "figureS3B_shared": {
        "model_type": "shared",
        "date_train": "2024_05_01",
        "date_test": "2024_05_02",
        "cs_lev": [1, 1], # level 2
        "ntest": 500,
        "imageset": 'bsd500',
        "dim": 2,
        "n_theta": 9,
        "n_row": 9,
        "n_col": 9,
        "tcov_loc": 37,
        "ncov_loc": 40,
        "tr_cov_scale": 1,
        "noise_cov_scale_pair": 0.5,
        "noise_cov_scale_single": 1,
        "ndraw": 500,
        "ntune": 500,
        "ncov_type": 'overlapped',
        "dc_offset": 2,
    },
    
    "figureS3C_shared": {
        "model_type": "shared",
        "date_train": "2024_05_01",
        "date_test": "2024_05_02",
        "cs_lev": [1, 1], # level 2
        "ntest": 500,
        "imageset": 'bsd500',
        "dim": 2,
        "n_theta": 9,
        "n_row": 9,
        "n_col": 9,
        "tcov_loc": 40,
        "ncov_loc": 37,
        "tr_cov_scale": 1,
        "noise_cov_scale_pair": 0.5,
        "noise_cov_scale_single": 1,
        "ndraw": 500,
        "ntune": 500,
        "ncov_type": 'overlapped',
        "dc_offset": 2,
    },
    
    "figureS3D_shared": {
        "model_type": "shared",
        "date_train": "2024_05_01",
        "date_test": "2024_05_02",
        "cs_lev": [1, 1], # level 2
        "ntest": 500,
        "imageset": 'bsd500',
        "dim": 2,
        "n_theta": 9,
        "n_row": 9,
        "n_col": 9,
        "tcov_loc": 37,
        "ncov_loc": 37,
        "tr_cov_scale": 1,
        "noise_cov_scale_pair": 0.5,
        "noise_cov_scale_single": 1,
        "ndraw": 500,
        "ntune": 500,
        "ncov_type": 'overlapped',
        "dc_offset": 2,
    },
    
    "figureS3A_ind": {
        "model_type": "ind",
        "date_train": "2024_05_01",
        "date_test": "2024_05_02",
        "cs_lev": [1, 1], # level 2
        "ntest": 500,
        "imageset": 'bsd500',
        "dim": 2,
        "n_theta": 9,
        "n_row": 9,
        "n_col": 9,
        "tcov_loc": 40,
        "ncov_loc": 40,
        "tr_cov_scale": 1,
        "noise_cov_scale_pair": 0.5,
        "noise_cov_scale_single": 1,
        "ndraw": 500,
        "ntune": 500,
        "ncov_type": 'overlapped',
        "dc_offset": 2,
    },
    
    "figureS3B_ind": {
        "model_type": "ind",
        "date_train": "2024_05_01",
        "date_test": "2024_05_02",
        "cs_lev": [1, 1], # level 2
        "ntest": 500,
        "imageset": 'bsd500',
        "dim": 2,
        "n_theta": 9,
        "n_row": 9,
        "n_col": 9,
        "tcov_loc": 37,
        "ncov_loc": 40,
        "tr_cov_scale": 1,
        "noise_cov_scale_pair": 0.5,
        "noise_cov_scale_single": 1,
        "ndraw": 500,
        "ntune": 500,
        "ncov_type": 'overlapped',
        "dc_offset": 2,
    },
    
    "figureS3C_ind": {
        "model_type": "ind",
        "date_train": "2024_05_01",
        "date_test": "2024_05_02",
        "cs_lev": [1, 1], # level 2
        "ntest": 500,
        "imageset": 'bsd500',
        "dim": 2,
        "n_theta": 9,
        "n_row": 9,
        "n_col": 9,
        "tcov_loc": 40,
        "ncov_loc": 37,
        "tr_cov_scale": 1,
        "noise_cov_scale_pair": 0.5,
        "noise_cov_scale_single": 1,
        "ndraw": 500,
        "ntune": 500,
        "ncov_type": 'overlapped',
        "dc_offset": 2,
    },
    
    "figureS3D_ind": {
        "model_type": "ind",
        "date_train": "2024_05_01",
        "date_test": "2024_05_02",
        "cs_lev": [1, 1], # level 2
        "ntest": 500,
        "imageset": 'bsd500',
        "dim": 2,
        "n_theta": 9,
        "n_row": 9,
        "n_col": 9,
        "tcov_loc": 37,
        "ncov_loc": 37,
        "tr_cov_scale": 1,
        "noise_cov_scale_pair": 0.5,
        "noise_cov_scale_single": 1,
        "ndraw": 500,
        "ntune": 500,
        "ncov_type": 'overlapped',
        "dc_offset": 2,
    },
    
    # Figure 3
    "figure03_shared": {
        "model_type": "shared",
        "date_train": "2024_05_01",
        "date_test": "2024_05_02",
        "cs_lev": [1, 1], # level 2
        "ntest": 500,
        "imageset": 'bsd500',
        "dim": 2,
        "n_theta": 9,
        "n_row": 9,
        "n_col": 9,
        "tcov_loc": 40,
        "ncov_loc": 40,
        "tr_cov_scale": 1,
        "noise_cov_scale_pair": 0.01,
        "noise_cov_scale_single": 1,
        "ndraw": 500,
        "ntune": 500,
        "ncov_type": 'overlapped',
        "dc_offset": 0,
    },
    
    "figure03_ind": {
        "model_type": "ind",
        "date_train": "2024_05_01",
        "date_test": "2024_05_02",
        "cs_lev": [1, 1], # level 2
        "ntest": 500,
        "imageset": 'bsd500',
        "dim": 2,
        "n_theta": 9,
        "n_row": 9,
        "n_col": 9,
        "tcov_loc": 37,
        "ncov_loc": 40,
        "tr_cov_scale": 1,
        "noise_cov_scale_pair": 1,
        "noise_cov_scale_single": 1,
        "ndraw": 500,
        "ntune": 500,
        "ncov_type": 'overlapped',
        "dc_offset": 0,
    },
    
    # Figure 6
    "figure06B_ind": {
        "model_type": "ind",
        "date_train": "2024_06_22",
        "date_test": "2024_06_22",
        "cs_lev": [1, 1], # level 2
        "ntest": 500, 
        "imageset": 'bsd500',
        "dim": 1,
        "n_theta": 15,
        "n_row": 1,
        "n_col": 17,
        "tcov_loc": 0,
        "ncov_loc": 0,
        "tr_cov_scale": 1,
        "noise_cov_scale_pair": 1,
        "noise_cov_scale_single": 1,
        "ndraw": 500, 
        "ntune": 500, 
        "ncov_type": 'overlapped',
        "dc_offset": 2,
    },
    
    "figure06B_shared": {
        "model_type": "shared",
        "date_train": "2024_06_22",
        "date_test": "2024_06_22",
        "cs_lev": [1, 1], # level 2
        "ntest": 500, 
        "imageset": 'bsd500',
        "dim": 1,
        "n_theta": 15,
        "n_row": 1,
        "n_col": 17,
        "tcov_loc": 0,
        "ncov_loc": 0,
        "tr_cov_scale": 1,
        "noise_cov_scale_pair": 0.01,
        "noise_cov_scale_single": 1,
        "ndraw": 500, 
        "ntune": 500, 
        "ncov_type": 'nonoverlapped',
        "dc_offset": 2,
    },
    
    # Supplementary Figure 2
    "figureS2_shared": {
        "model_type": "shared",
        "date_train": "2024_05_01",
        "date_test": "2024_05_02",
        "cs_lev": [1, 1], # level 2
        "ntest": 500,
        "imageset": 'bsd500',
        "dim": 2,
        "n_theta": 9,
        "n_row": 9,
        "n_col": 9,
        "tcov_loc": 40,
        "ncov_loc": 40,
        "tr_cov_scale": 1,
        "noise_cov_scale_pair": 1,
        "noise_cov_scale_single": 1,
        "ndraw": 500,
        "ntune": 500,
        "ncov_type": 'overlapped',
        "dc_offset": 0,
    },

    "figureS2_ind": {
        "model_type": "ind",
        "date_train": "2024_05_01",
        "date_test": "2024_05_02",
        "cs_lev": [1, 1], # level 2
        "ntest": 500,
        "imageset": 'bsd500',
        "dim": 2,
        "n_theta": 9,
        "n_row": 9,
        "n_col": 9,
        "tcov_loc": 37,
        "ncov_loc": 40,
        "tr_cov_scale": 1,
        "noise_cov_scale_pair": 1,
        "noise_cov_scale_single": 1,
        "ndraw": 500,
        "ntune": 500,
        "ncov_type": 'overlapped',
        "dc_offset": 0,
    },

    # Supplementary Figure 13
    "figureS13-panelA-middle": {
        "model_type": "shared",
        "date_train": "2024_05_01",
        "date_test": "2024_05_02",
        "cs_lev": [1, 1], # level 2
        "ntest": 500,
        "imageset": 'bsd500',
        "dim": 2,
        "n_theta": 9,
        "n_row": 9,
        "n_col": 9,
        "tcov_loc": 40,
        "ncov_loc": 40,
        "tr_cov_scale": 1,
        "noise_cov_scale_pair": 0.01,
        "noise_cov_scale_single": 1,
        "ndraw": 500,
        "ntune": 500,
        "ncov_type": 'overlapped',
        "dc_offset": 0,
    },
    
    "figureS13-panelA-right": {
        "model_type": "ind",
        "date_train": "2024_05_01",
        "date_test": "2024_05_02",
        "cs_lev": [1, 1], # level 2
        "ntest": 500,
        "imageset": 'bsd500',
        "dim": 2,
        "n_theta": 9,
        "n_row": 9,
        "n_col": 9,
        "tcov_loc": 37,
        "ncov_loc": 40,
        "tr_cov_scale": 1,
        "noise_cov_scale_pair": 1,
        "noise_cov_scale_single": 1,
        "ndraw": 500,
        "ntune": 500,
        "ncov_type": 'overlapped',
        "dc_offset": 0,
    },
    
    "figureS13-panelB-middle": {
        "model_type": "shared",
        "date_train": "2024_05_02",
        "date_test": "2024_05_03",
        "cs_lev": [2, 2], # level 3
        "ntest": 500,
        "imageset": 'bsd500',
        "dim": 2,
        "n_theta": 9,
        "n_row": 9,
        "n_col": 9,
        "tcov_loc": 40,
        "ncov_loc": 40,
        "tr_cov_scale": 1,
        "noise_cov_scale_pair": 0.01,
        "noise_cov_scale_single": 1,
        "ndraw": 500,
        "ntune": 500,
        "ncov_type": 'overlapped',
        "dc_offset": 0,
    },
    
    "figureS13-panelB-right": {
        "model_type": "ind",
        "date_train": "2024_05_02",
        "date_test": "2024_05_03",
        "cs_lev": [2, 2], # level 3
        "ntest": 500,
        "imageset": 'bsd500',
        "dim": 2,
        "n_theta": 9,
        "n_row": 9,
        "n_col": 9,
        "tcov_loc": 37,
        "ncov_loc": 40,
        "tr_cov_scale": 1,
        "noise_cov_scale_pair": 1,
        "noise_cov_scale_single": 1,
        "ndraw": 500,
        "ntune": 500,
        "ncov_type": 'overlapped',
        "dc_offset": 0,
    },
    
    "figureS13-panelC-middle": {
        "model_type": "shared",
        "date_train": "2024_05_01",
        "date_test": "2024_05_29",
        "cs_lev": [1, 1], # level 1
        "ntest": 270,
        "imageset": 'nn2015',
        "dim": 2,
        "n_theta": 9,
        "n_row": 9,
        "n_col": 9,
        "tcov_loc": 40,
        "ncov_loc": 40,
        "tr_cov_scale": 1,
        "noise_cov_scale_pair": 0.01,
        "noise_cov_scale_single": 1,
        "ndraw": 500,
        "ntune": 500,
        "ncov_type": 'overlapped',
        "dc_offset": 0,
    },
    
    "figureS13-panelC-right": {
        "model_type": "ind",
        "date_train": "2024_05_01",
        "date_test": "2024_05_29",
        "cs_lev": [1, 1], # level 1
        "ntest": 270,
        "imageset": 'nn2015',
        "dim": 2,
        "n_theta": 9,
        "n_row": 9,
        "n_col": 9,
        "tcov_loc": 37,
        "ncov_loc": 40,
        "tr_cov_scale": 1,
        "noise_cov_scale_pair": 1,
        "noise_cov_scale_single": 1,
        "ndraw": 500,
        "ntune": 500,
        "ncov_type": 'overlapped',
        "dc_offset": 0,
    },
    
    # Supplementary Figure 16
    "figureS16_shared": {
        "model_type": "shared",
        "date_train": "2024_05_01",
        "date_test": "2024_05_02",
        "cs_lev": [1, 1], # level 2
        "ntest": 500,
        "imageset": 'bsd500',
        "dim": 2,
        "n_theta": 9,
        "n_row": 9,
        "n_col": 9,
        "tcov_loc": 40,
        "ncov_loc": 40,
        "tr_cov_scale": 1,
        "noise_cov_scale_pair": 0.01,
        "noise_cov_scale_single": 1,
        "ndraw": 5000,
        "ntune": 500,
        "ncov_type": 'overlapped',
        "dc_offset": 0,
    },
    
    "figureS16_ind": {
        "model_type": "ind",
        "date_train": "2024_05_01",
        "date_test": "2024_05_02",
        "cs_lev": [1, 1], # level 2
        "ntest": 500,
        "imageset": 'bsd500',
        "dim": 2,
        "n_theta": 9,
        "n_row": 9,
        "n_col": 9,
        "tcov_loc": 37,
        "ncov_loc": 40,
        "tr_cov_scale": 1,
        "noise_cov_scale_pair": 1,
        "noise_cov_scale_single": 1,
        "ndraw": 5000,
        "ntune": 500,
        "ncov_type": 'overlapped',
        "dc_offset": 0,
    },
}

In [ ]:
# function to apply settings for a given figure-panel key
def run_experiment(key):
    setting = settings[key]
    date_train = setting["date_train"]
    date_test = setting["date_test"]
    cs_lev = setting["cs_lev"]
    ntest = setting["ntest"]
    imageset = setting["imageset"]
    dim = setting["dim"]
    n_theta = setting["n_theta"]
    n_row_col = setting["n_col"]
    n_loc = setting["n_row"] * setting["n_col"]
    tr_cov_scale = setting["tr_cov_scale"]
    noise_cov_scale_pair = setting["noise_cov_scale_pair"]
    noise_cov_scale_single = setting["noise_cov_scale_single"]
    tcov_loc = setting["tcov_loc"]  
    ncov_loc = setting["ncov_loc"]
    model_type = setting["model_type"]
    ndraw = setting["ndraw"]
    ntune = setting["ntune"]
    ncov_type = setting["ncov_type"]
    dc_offset = setting["dc_offset"]
    
    # your code here using the settings above
    print(
    f"Running {key}:\n"
    f" train date:         {date_train}\n"
    f" test date:          {date_test}\n"
    f" cs levels:          {cs_lev}\n"
    f" ntest:              {ntest}\n"
    f" imageset:           {imageset}\n"
    f" dim:                {dim}\n"
    f" n_theta:            {n_theta}\n"
    f" n_row_col:          {n_row_col}\n"
    f" n_loc:              {n_loc}\n"
    f" tr_cov_scale:       {tr_cov_scale}\n"
    f" ncov_scale_pair:    {noise_cov_scale_pair}\n"
    f" ncov_scale_single:  {noise_cov_scale_single}\n"
    f" tcov_loc:               {tcov_loc}\n"
    f" ncov_loc:               {ncov_loc}\n"
    f" mode_type:          {model_type}\n"
    f" ndraw:              {ndraw}\n"
    f" ntune:              {ntune}\n"
    f" ncov type:          {ncov_type}\n"
    f" dc_offset:          {dc_offset}\n"
)
    
    return model_type, cs_lev, date_train, date_test, ntest, imageset, dim, n_theta, n_row_col, n_loc, tr_cov_scale, noise_cov_scale_pair, noise_cov_scale_single, tcov_loc, ncov_loc, ndraw, ntune, ncov_type, dc_offset

In [ ]:
# running settings
fig_name = "figureS16_shared" 
model_type,
cs_lev,
date_train,
date_test,
ntest,
imageset,
dim,
n_theta,
n_row_col,
n_loc,
tr_cov_scale,
noise_cov_scale_pair,
noise_cov_scale_single,
tcov_loc,
ncov_loc,
ndraw,
ntune,
ncov_type,
dc_offset = run_experiment(fig_name)

In [ ]:
# settings

# images and filters settings
ncent = 2
nsurr = 8
nphase = 2
nfilt = 36
alpha = 1
cond_name = f'nloc_{n_loc}_ntheta_{n_theta}' # name of condition

# directory paths
file_path = '/home/csv_files/'

In [ ]:
# load covariance matrix
with open(f'{file_path}cov_mat_dim_{dim}_level_{cs_lev[0]}_{cond_name}_date_{date_train}.csv', "rb") as fp:
    cov_mat_dict = pickle.load(fp)  
train_cov_mat = cov_mat_dict['train_cov_mat']
noise_cov_mat = cov_mat_dict['noise_cov_mat']

# load response to test natural images
with open(f'{file_path}test_res_{imageset}_dim_{dim}_level_{cs_lev[0]}_{cond_name}_date_{date_test}.csv', "rb") as fp:   # Unpickling
    test_res_dict = pickle.load(fp)

test_filter_sz_res_conds = test_res_dict['test_filter_res']

In [ ]:
# covariance matrix scaling mask
mask = np.ones((nfilt,nfilt)) * noise_cov_scale_single
mask[:int(nfilt/2),int(nfilt/2):] = noise_cov_scale_pair
mask[int(nfilt/2):,:int(nfilt/2)] = noise_cov_scale_pair

# scale covariance matrices
train_cov_mat = train_cov_mat * tr_cov_scale
noise_cov_mat = noise_cov_mat * mask

In [ ]:
# selected image index
img_idx = np.arange(0, ntest)

# reshape test response according to selected images
test_filter_sz_res = np.transpose(test_filter_sz_res_conds[:,tcov_loc,img_idx,:],(0,2,1))

# sorting images based on the center 1 filter response (high to low)
s_idx = np.argsort(test_filter_sz_res[0,0,:])[::-1]
img_idx = s_idx[0:ntest]
test_filter_sz_res = test_filter_sz_res[:,:,img_idx]

Define prior of random variables


In [ ]:
# define mean and covariance of priors
g_mu = np.zeros(ncent * (nsurr + 1) * nphase)

# two neurons indexes
ind_n1 = np.linspace(0, 17, 18).astype(int) # neuron 1
ind_n2 = np.linspace(18, 35, 18).astype(int) # neuron 2
gs_small_large = [[],[],[],[]]

Gaussian Scale Mixture model

In [ ]:
# loop over each theta- small images
for i in range(n_theta):    
    g_cov = train_cov_mat[tcov_loc, i, :, :]
    n_cov = noise_cov_mat[ncov_loc, i, :, :]
    idx = np.linspace(i * nfilt, (i + 1) * nfilt - 1, nfilt).astype(int)
    data1 = test_filter_sz_res[0, idx, :]
    data1 = np.squeeze(data1)
    data1 = np.transpose(data1)

    print('Size of observation matrix: (image by filter)', data1.shape)
    
    with pm.Model() as GSM_small:
        if model_type == "ind":
            # Independent model specifics
            gs1 = pm.MvNormal('gs1', mu=g_mu[ind_n1], cov=g_cov[np.ix_(ind_n1, ind_n1)], shape=(ntest, (nsurr + 1) * nphase))
            gs2 = pm.MvNormal('gs2', mu=g_mu[ind_n2], cov=g_cov[np.ix_(ind_n2, ind_n2)], shape=(ntest, (nsurr + 1) * nphase))
            vs1 = pm.Weibull('vs1', 2, np.sqrt(2), shape=ntest) 
            vs2 = pm.Weibull('vs2', 2, np.sqrt(2), shape=ntest)
            mu_1 = vs1 * gs1.T
            mu_2 = vs2 * gs2.T
            mu_ = pm.math.concatenate((mu_1.T, mu_2.T), axis=1)
        else:
            # Shared model specifics
            gs = pm.MvNormal('gs', mu=g_mu, cov=g_cov, shape=(ntest, ncent * (nsurr + 1) * nphase))
            vs = pm.Weibull('vs', 2, np.sqrt(2), shape=ntest)  
            mu_ = (vs * gs.T).T
        
        xs = pm.MvNormal('xs', mu=mu_, cov=n_cov, observed=data1) 

        # Sampling
        with GSM_small:
            step = pm.NUTS(target_accept=0.9)
            samples_small = pm.sample(draws=ndraw, tune=ntune, step=step, random_seed=42, return_inferencedata=False, cores=4, chains=4)
            
        if model_type == "ind":
            gs_small_large[0].append(samples_small['gs1'])  # Neuron 1 small size
            gs_small_large[2].append(samples_small['gs2'])  # Neuron 2 small size
        else:
            gs_small_large[0].append(samples_small['gs'][:,:,ind_n1])  # Neuron 1
            gs_small_large[2].append(samples_small['gs'][:,:,ind_n2])  # Neuron 2

In [ ]:
# loop over each theta- large images
for i in range(n_theta):
    g_cov = train_cov_mat[tcov_loc, i, :, :]
    n_cov = noise_cov_mat[ncov_loc, i, :, :]
    idx = np.linspace((i * nfilt), ((i + 1) * nfilt) - 1, nfilt).astype(int)

    data2 = test_filter_sz_res[1, idx, :]
    data2 = np.squeeze(data2)
    data2 = np.transpose(data2)
    print('Size of observation matrix: (image by filter)', data2.shape)

    with pm.Model() as GSM_large:
        if model_type == "ind":
            # Independent model specifics
            gl1 = pm.MvNormal('gl1', mu=g_mu[ind_n1], cov=g_cov[np.ix_(ind_n1, ind_n1)], shape=(ntest, (nsurr + 1) * nphase))
            gl2 = pm.MvNormal('gl2', mu=g_mu[ind_n2], cov=g_cov[np.ix_(ind_n2, ind_n2)], shape=(ntest, (nsurr + 1) * nphase))
            vl1 = pm.Weibull('vl1', 2, np.sqrt(2), shape=ntest) 
            vl2 = pm.Weibull('vl2', 2, np.sqrt(2), shape=ntest)
            mu_1 = vl1 * gl1.T
            mu_2 = vl2 * gl2.T
            mu_ = pm.math.concatenate((mu_1.T, mu_2.T), axis=1)
        else:
            # Shared model specifics
            gl = pm.MvNormal('gl', mu=g_mu, cov=g_cov, shape=(ntest, ncent * (nsurr + 1) * nphase))
            vl = pm.Weibull('vl', 2, np.sqrt(2), shape=ntest) 
            mu_ = (vl * gl.T).T
        
        xl = pm.MvNormal('xl', mu=mu_, cov=n_cov, observed=data2)

        # Sampling
        with GSM_large:
            step = pm.NUTS(target_accept=0.9)
            samples_large = pm.sample(draws=ndraw, tune=ntune, step=step, random_seed=42, return_inferencedata=False, cores=4, chains=4)

        # Store results
        if model_type == "ind":
            gs_small_large[1].append(samples_large['gl1'])  # Neuron 1 large size
            gs_small_large[3].append(samples_large['gl2'])  # Neuron 2 large size
        else:
            gs_small_large[1].append(samples_large['gl'][:,:,ind_n1])  # Neuron 1
            gs_small_large[3].append(samples_large['gl'][:,:,ind_n2])  # Neuron 2

In [ ]:
# Computing model neuron spike count
spike_counts = [[] for j in range(n_theta)]

# Loop over orientation preferences
for i_d_theta in range(0, n_theta):
    tmp_cond = []
    
    # Loop over model neurons
    for i_neuron in range(0, ncent * nphase):  # Two small and two large model neuron responses
        tmp = []
        
        # Loop over test images
        for i_image in range(0, ntest):
            # Get Gaussian samples for center 1 and center 2
            g1 = gs_small_large[i_neuron][i_d_theta][:, i_image, 0] + dc_offset # Center 1
            g2 = gs_small_large[i_neuron][i_d_theta][:, i_image, 9] + dc_offset # Center 2
            
            # Rectify the responses (set negative values to 0)
            g1 = np.maximum(g1, 0)
            g2 = np.maximum(g2, 0)
            
            # Compute spike count as the sum of responses multiplied by a factor alpha
            spike_count = alpha * (g1 + g2)
            tmp.append(spike_count)
        
        tmp_cond.append(tmp)
    
    # Store spike counts for the current orientation preference
    spike_counts[i_d_theta] = np.array(tmp_cond, dtype=object)

In [ ]:
# Computing noise correlation per image
rsc_images = np.zeros((n_theta, 2, ntest))

for i_d_theta in range(n_theta):
    for i_size in range(2):  # Loop over small and large sizes
        for i_image in range(ntest):  # Loop over test images
            # Compute Pearson correlation coefficient between spike counts of two neurons
            # of the same size for the current orientation preference and image
            stat, _ = pearsonr(spike_counts[i_d_theta][i_size, i_image, :],
                               spike_counts[i_d_theta][i_size + 2, i_image, :])
            rsc_images[i_d_theta, i_size, i_image] = stat

In [ ]:
# save files
with open(file_name, "wb") as fp:  
    pickle.dump({
        'gs': gs_small_large,
        'sample_small': sample_g_v_small,
        'sample_large': sample_g_v_large,
        'rsc': rsc_images,
        'spike_count': spike_counts
    }, fp)